In [ ]:
#!pip install --quiet bs4
from bs4 import BeautifulSoup 
import requests
import csv
import pandas as pd
from time import time

In [ ]:
def crawl_page(i): # Hàm crawl dữ liệu 
  stri = f'https://nha.chotot.com/toan-quoc/mua-ban-nha-dat?page={i}&fbclid=IwAR3WwHSC9jKqOASOzJRJwj4_H_2Fh5pjC4gklRvYBGwu7-5t3JE_DbYoNMQ' 
  batch = []
  try:
    source = requests.get(stri).text
    soup = BeautifulSoup(source, 'lxml')
    a = soup.find_all('div', class_="list-view")
    for i in a: #for 2nd
      try:
        a1 = i.find_all('div', class_ ="ListAds_ListAds__1z6Pv col-xs-12 no-padding")
        for i in a1: #for 3rd
          try:
            a2 = i.ul.find_all('div', role="button", tabindex="0")
            for i in a2: #for 4th
              try:
                string1 = i.li.a['href'].split('/')
                #string2 = string1[4].split('.')
                #string3 = 'mua-ban-dat'
                #link = "f'https://nha.chotot.com/{string1[1]}/{string1[2]/{string3}/{string2[0]}}'"
                #link = "https://nha.chotot.com/" + string1[1] + "/" +  string1[2] + "/" + "mua-ban-dat" + string2[0] +".htm"
                link = "https://nha.chotot.com/" + "mua-ban-nha-dat" + string1[1].split('[object Object]')[1] + "/" + string1[2]
                
                source = requests.get(link).text
                soup = BeautifulSoup(source, 'lxml')
                a = soup.find('div', class_="col-xs-12 no-padding")
                if type(a) == "NoneType":
                    continue
                b = a.find_all('div', class_ ="col-md-6 no-padding AdParam_adParamItem__1o-dd")
                if type(b) == "NoneType":
                    continue
                dien_tich_dat = "NaN"
                gia_m2 = "NaN"
                so_phong_ngu = "NaN"
                so_phong_ve_sinh = "NaN"
                so_tang = "NaN"
                giay_to_phap_ly = "NaN"
                chieu_ngang = "NaN"
                chieu_doc = "NaN"
                dien_tich_su_dung = "NaN"
                tinh_trang_noi_that ="NaN"
                loai_hinh_nha_o = "NaN"
                dac_diem_nha_dat = "NaN"
                huong_cua_chinh = "NaN"
                
                #crawl tọa độ
                a1 = soup.find("script", id="__NEXT_DATA__", type="application/json")
                b1 = str(a1)
                x = b1.split('longitude":')
                #xử lý ngoại lệ
                try:
                    x1 = x[1].split(',')
                    lon = x[1].split(',')[0]
                    lat = x1[1].split('latitude":')[1]
                except:
                    lon = "NaN"
                    lat = "NaN"
                
                tinh_thanh = b1.split('"area_name":"')[1].split('"')[0]
                quan_huyen = b1.split('"region_name":"')[1].split('"')[0]
                
                for i in b: 
                    c = i.find('div', class_="media-body media-middle")
                    d = c.find('span', class_="AdParam_adParamValue__1ayWO")

                    if d['itemprop'] == "size": 
                        u = d.text.split(' ')
                        dien_tich_dat = u[0]
                    if d['itemprop'] == "price_m2":
                        u = d.text.split(' ')
                        gia_m2 = u[0]
                    if d['itemprop'] == "rooms": 
                        u = d.text.split(' ')
                        so_phong_ngu = u[0]
                    if d['itemprop'] == "toilets":
                        u = d.text.split(' ')
                        so_phong_ve_sinh = u[0]
                    if d['itemprop'] == "floors": 
                        so_tang = d.text
                    if d['itemprop'] == "property_legal_document": 
                        giay_to_phap_ly = d.text
                    if d['itemprop'] == "width": 
                        chieu_ngang = d.text
                    if d['itemprop'] == "width": 
                        chieu_ngang = d.text
                    if d['itemprop'] == "length": 
                        chieu_doc = d.text
                    if d['itemprop'] == "living_size":
                        u = d.text.split(' ')
                        dien_tich_su_dung = u[0]
                    if d['itemprop'] == "furnishing_sell": 
                        tinh_trang_noi_that = d.text    
                    if d['itemprop'] == "house_type": 
                        loai_hinh_nha_o = d.text
                    if d['itemprop'] == "property_road_condition": 
                        dac_diem_nha_dat = d.text
                    if d['itemprop'] == "direction": 
                        huong_cua_chinh = d.text
                batch.append((dien_tich_dat, gia_m2, tinh_thanh, quan_huyen, so_phong_ngu, so_phong_ve_sinh, so_tang, giay_to_phap_ly, chieu_ngang, chieu_doc, dien_tich_su_dung, lon, lat, tinh_trang_noi_that, loai_hinh_nha_o, dac_diem_nha_dat, huong_cua_chinh))
              except:
                num_err += 1
                print('for 4th error!')
                continue
          except:
            num_err += 1
            print('for 3rd error!')
            continue
      except:
        num_err += 1
        print('for 2nd error!')
        continue     
  except:
    num_err += 1
    print('for 1st error!')
  finally:
    return batch

In [ ]:
# Do not run this cell. Prefer running the cell below.
A = []
num_err = 0
start = time()
print('Start crawling data!')

for i in range(10,20): #for 1st
  A += crawl_page(i) # Tính toán đơn luồng

print('Done!')
column_name = ['Diện tích đất', 'Giá/m2', 'Tỉnh thành', 'Quận huyện' ,'Số phòng ngủ', 'Số phòng vệ sinh', 'Số tầng', 'Giấy tờ pháp lý', 'Chiều ngang', 'Chiều dọc', 'Diện tích sử dụng', 'Kinh độ', 'Vĩ độ', 'Tình trạng nội thất', 'Loại hình nhà ở', 'Đặc điểm nhà đất', 'Hướng cửa chính']
df = pd.DataFrame.from_records(A, columns=column_name)
filename = 'data10-20.csv'
df.to_csv(filename)
print('Saved to file: ', filename)
print('Crawling time: ', time()-start)
print('Number of records: ', len(df))
if num_err:
  print('Crawling errors: ', num_err)

Start crawling data!
Done!
Saved to file:  data10-20.csv
Crawling time:  297.22533226013184
Number of records:  200


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
A = []
num_err = 0
start = time()
print('Start crawling data!')
processes = []
with ThreadPoolExecutor(max_workers=200) as executor: # Tính toán đa luồng
  for i in range(6000, 7000, 3): #for 1st
    processes.append(executor.submit(lambda i: crawl_page(i) + crawl_page(i+1) + crawl_page(i+2), i)) # crawl 3 page mỗi luồng

for task in as_completed(processes):
  A += task.result()
print('Done!')
column_name = ['Diện tích đất', 'Giá/m2', 'Tỉnh thành', 'Quận huyện' ,
               'Số phòng ngủ', 'Số phòng vệ sinh', 'Số tầng', 'Giấy tờ pháp lý',
               'Chiều ngang', 'Chiều dọc', 'Diện tích sử dụng', 'Kinh độ',
               'Vĩ độ', 'Tình trạng nội thất', 'Loại hình nhà ở', 'Đặc điểm nhà đất', 'Hướng cửa chính']
df = pd.DataFrame.from_records(A, columns=column_name)
filename = 'data6000-7000.csv'
df.to_csv(filename)
print('Saved to file: ', filename)
print('Crawling time: ', time()-start)
print('Number of records: ', len(df))
if num_err:
  print('Crawling errors: ', num_err)

Start crawling data!
Done!
Saved to file:  data6000-7000.csv
Crawling time:  1364.511741399765
Number of records:  16292


In [ ]:
######################################################
import pandas as pd ### ghép với mật độ dân số
import numpy as np
pop = pd.read_csv('table (1).csv') #file mật độ dân số theo quận huyện
df = pd.read_csv('data2-7000.csv')#, encoding='utf-8')

In [ ]:
df.rename(columns={'Tỉnh thành': 'Quận huyện', 'Quận huyện': 'Tỉnh thành'}, inplace=True)

In [ ]:
df['Mật độ dân số'] = np.nan

In [ ]:
popdict = {}
for x in pop['tỉnh thành'].unique():
  popdict[x] = {}
for x in pop.itertuples():
  popdict[x[3]][x[2]] = x[4]
popdict['Tp Hồ Chí Minh'] = popdict['Thành phố Hồ Chí Minh']
popdict['Tp Hồ Chí Minh']['Quận 2'] = popdict['Tp Hồ Chí Minh']['Thủ Đức']
popdict['Tp Hồ Chí Minh']['Quận 9'] = popdict['Tp Hồ Chí Minh']['Thủ Đức']
popdict['Bà Rịa - Vũng Tàu'] = popdict['Bà Rịa – Vũng Tàu']
print(popdict)
!pip install --quiet unidecode
from unidecode import unidecode
err = []
for x in df.index:
  a = popdict[df.loc[x, 'Tỉnh thành']]
  for i in a.keys():
    ix = unidecode(i.lower())
    if ix in unidecode(df.loc[x, 'Quận huyện'].lower()):
      df.loc[x, 'Mật độ dân số'] = a[i]
      break
    err.append(i) # bản ghi ko điền được mật độ dân số


{'Kiên Giang': {'An Biên': 288, 'An Minh': 196, 'Châu Thành': 559, 'Giang Thành': 71, 'Giồng Riềng': 351, 'Gò Quao': 301, 'Hà Tiên': 812, 'Hòn Đất': 150, 'Kiên Hải': 715, 'Kiên Lương': 168, 'Phú Quốc': 305, 'Rạch Giá': 2196, 'Tân Hiệp': 297, 'U Minh Thượng': 147, 'Vĩnh Thuận': 208}, 'Hải Phòng': {'An Dương': 1878, 'An Lão': 1246, 'Bạch Long Vĩ': 201, 'Cát Hải': 99, 'Đồ Sơn': 1068, 'Dương Kinh': 1289, 'Hải An': 1282, 'Hồng Bàng': 6628, 'Kiến An': 3988, 'Kiến Thụy': 1289, 'Lê Chân': 18467, 'Ngô Quyền': 14629, 'Thủy Nguyên': 1275, 'Tiên Lãng': 800, 'Vĩnh Bảo': 997}, 'Gia Lai': {'An Khê': 329, 'Ayun Pa': 139, 'Chư Păh': 79, 'Chư Prông': 73, 'Chư Pưh': 108, 'Chư Sê': 190, 'Đak Đoa': 125, 'Đak Pơ': 80, 'Đức Cơ': 105, 'Ia Grai': 94, 'Ia Pa': 65, 'Kbang': 36, 'Kông Chro': 36, 'Krông Pa': 53, 'Mang Yang': 61, 'Phú Thiện': 156, 'Pleiku': 977}, 'Bình Định': {'An Lão': 40, 'An Nhơn': 719, 'Hoài Ân': 114, 'Hoài Nhơn': 504, 'Phù Cát': 269, 'Phù Mỹ': 291, 'Quy Nhơn': 1014, 'Tây Sơn': 168, 'Tuy Phước'

In [ ]:
print(df['Tỉnh thành'].unique())
print(pop['tỉnh thành'].unique())

['Đà Nẵng' 'Tp Hồ Chí Minh' 'Cần Thơ' 'Hà Nội' 'Bình Dương' 'Đồng Nai'
 'Hải Phòng' 'Long An' 'Quảng Ngãi' 'Đắk Lắk' 'Bắc Giang' 'Khánh Hòa'
 'Bến Tre' 'Thừa Thiên Huế' 'Sóc Trăng' 'Tây Ninh' 'Vĩnh Phúc' 'Hưng Yên'
 'Bình Thuận' 'Kiên Giang' 'An Giang' 'Bắc Ninh' 'Quảng Nam' 'Nam Định'
 'Vĩnh Long' 'Lâm Đồng' 'Đắk Nông' 'Bình Phước' 'Cà Mau'
 'Bà Rịa - Vũng Tàu' 'Đồng Tháp' 'Hòa Bình' 'Bình Định' 'Hải Dương'
 'Ninh Thuận' 'Tiền Giang' 'Thái Nguyên' 'Thanh Hóa' 'Nghệ An' 'Hậu Giang'
 'Phú Yên' 'Bạc Liêu' 'Quảng Ninh' 'Lào Cai' 'Trà Vinh' 'Phú Thọ'
 'Gia Lai' 'Thái Bình' 'Hà Tĩnh' 'Ninh Bình' 'Kon Tum' 'Hà Nam' 'Yên Bái'
 'Lạng Sơn' 'Sơn La' 'Quảng Trị' 'Tuyên Quang' 'Bắc Kạn' 'Hà Giang'
 'Quảng Bình']
['Kiên Giang' 'Hải Phòng' 'Gia Lai' 'Bình Định' 'An Giang' 'Hưng Yên'
 'Nghệ An' 'Bắc Kạn' 'Quảng Ninh' 'Hà Nội' 'Quảng Bình'
 'Bà Rịa – Vũng Tàu' 'Thanh Hóa' 'Quảng Ngãi' 'Bến Tre' 'Ninh Thuận'
 'Bình Thuận' 'Bắc Giang' 'Lào Cai' 'Bạc Liêu' 'Hà Giang' 'Bắc Ninh'
 'Lạng Sơn' 'Bình Dương' '

In [ ]:
df

,Unnamed: 0,Diện tích đất,Giá/m2,Quận huyện,Tỉnh thành,Số phòng ngủ,Số phòng vệ sinh,Số tầng,Giấy tờ pháp lý,Chiều ngang,Chiều dọc,Diện tích sử dụng,Kinh độ,Vĩ độ,Tình trạng nội thất,Loại hình nhà ở,Đặc điểm nhà đất,Hướng cửa chính,Mật độ dân số
0,0,75.0,72,Quận Liên Chiểu,Đà Nẵng,3,4,NaN,Đã có sổ,NaN,NaN,NaN,108.16904,16.07240,Nội thất cao cấp,Nhà mặt phố,NaN,Tây Bắc,2616.0
1,1,400.0,"62,5",Quận 8,Tp Hồ Chí Minh,10,Nhiều,NaN,Đã có sổ,16 m,25 m,NaN,106.62309,10.71289,NaN,Nhà mặt phố,Hẻm xe hơi,NaN,22118.0
2,2,10.0,96,Quận 11,Tp Hồ Chí Minh,2,2,2,Đã có sổ,4 m,3 m,11.0,106.65273,10.76385,NaN,Nhà ngõ,NaN,NaN,18523.0
3,3,56.0,"67,86",Quận Bình Tân,Tp Hồ Chí Minh,4,4,NaN,Đã có sổ,4 m,14 m,NaN,106.58543,10.79849,NaN,Nhà ngõ,Hẻm xe hơi,NaN,15109.0
4,4,85.0,"9,41",Quận Ô Môn,Cần Thơ,2,1,NaN,Đã có sổ,5 m,17 m,65.0,105.55076,10.18150,NaN,Nhà ngõ,Hẻm xe hơi,NaN,976.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129830,16287,90.0,"68,33",Quận Tân Phú,Tp Hồ Chí Minh,2,NaN,NaN,Đã có sổ,4.55 m,19.5 m,NaN,106.62910,10.81209,Nội thất đầy đủ,Nhà ngõ,Hẻm xe hơi,NaN,30146.0
129831,16288,315.0,"158,73",Quận Tân Bình,Tp Hồ Chí Minh,1,1,NaN,NaN,8 m,40 m,NaN,106.64857,10.80190,NaN,Nhà mặt phố,NaN,NaN,21196.0
129832,16289,45.0,"322,22",Quận Bắc Từ Liêm,Hà Nội,4,4,NaN,Đã có sổ,4.5 m,10 m,225.0,105.76359,21.05034,Nội thất cao cấp,Nhà biệt thự,NaN,NaN,7529.0
129833,16290,95.0,"65,26",Huyện Đan Phượng,Hà Nội,5,NaN,NaN,Đã có sổ,NaN,NaN,NaN,105.47380,21.16054,NaN,Nhà phố liền kề,NaN,NaN,2237.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129835 entries, 0 to 129834
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           129835 non-null  int64  
 1   Diện tích đất        129695 non-null  float64
 2   Giá/m2               129693 non-null  object 
 3   Quận huyện           129835 non-null  object 
 4   Tỉnh thành           129835 non-null  object 
 5   Số phòng ngủ         129830 non-null  object 
 6   Số phòng vệ sinh     117559 non-null  object 
 7   Số tầng              62439 non-null   object 
 8   Giấy tờ pháp lý      111371 non-null  object 
 9   Chiều ngang          104321 non-null  object 
 10  Chiều dọc            97026 non-null   object 
 11  Diện tích sử dụng    94710 non-null   float64
 12  Kinh độ              122279 non-null  float64
 13  Vĩ độ                122279 non-null  float64
 14  Tình trạng nội thất  58712 non-null   object 
 15  Loại hình nhà ở  

In [ ]:
df.to_csv('dataset.csv')